In [ ]:
%websocketconnect 192.168.1.130 --password horsetoe -kbi

In [ ]:
2+2

In [ ]:
global wdt_counter
wdt_counter = 0

![Pinout](nodemcu_pins.png)

![Pinout2](ESP8266-NodeMCU-kit-12-E-pinout-gpio-pin.png)

![Pin details](pin_details.png)

![Schematic](schematic.png)

### Photon pin - ESP8266 pin

D2 - D6 (Machine pin 12)  is rain gauge

D3 - D5 (Machine pin 14) is wind speed

A0 - A0 is wind vane

D1 - D1 / SCL
D0 - D2 / SDA

# Set up pins

## LED

In [ ]:
import machine

In [ ]:
frequency = 5000

In [ ]:
led = machine.PWM(machine.Pin(2), frequency)

In [ ]:
led.duty(0)

In [ ]:
led.duty(1023)

## Wind direction

#### Wind direction is hooked up to the analog to digital coverter on pin 0

In [ ]:
wind_direction_pin = machine.ADC(0)

In [ ]:
wind_direction_pin.read()

#### Translate the voltage read from the ADC (range 0-1024) to a cardinal direction

![Wind Vane Resistors](wind_vane_resistors.png)

Manually checked each direction, and these are the raw values I get

N = 825

NNW = 741

NW = 925

WNW = 867

W = 985

WSW = 634

SW = 666

SSW = 268

S = 314

SSE = 144

SE = 207

ESE = 77

E = 108

ENE = 97

NE = 495

NNE = 439

In [ ]:
vals = [825, 741, 925, 867, 985, 634, 666, 268, 314, 144, 207, 77, 108, 97, 495, 439]

In [ ]:
vals.sort()

In [ ]:
vals

In [ ]:
wind_direction_pin.read()

In [ ]:
wind_direction_dict = {(0, 60):    "Error.low",
                       (61, 85):   "ESE",
                       (86, 103):  "ENE",
                       (104, 130): "E",
                       (131, 180): "SSE",
                       (181, 240): "SE",
                       (241, 290): "SSW",
                       (291, 400): "S",
                       (401, 470): "NNE",
                       (471, 560): "NE",
                       (561, 650): "WSW",
                       (651, 700): "SW",
                       (701, 780): "NNW",
                       (781, 845): "N",
                       (846, 900): "WNW",
                       (901, 950): "NW",
                       (951, 1000):"W",
                       (1001, 1024):"Error.high"}

In [ ]:
def getDirection(table, raw):
    for key in table:
        if key[0] <= raw <= key[1]:
            return table[key]

In [ ]:
wind_direction_raw = wind_direction_pin.read()

In [ ]:
wind_direction_raw

In [ ]:
getDirection(wind_direction_dict, wind_direction_raw)

## Wind speed

Interrupts may be attached to any GPIO pin except GPIO16. Since GPIO6-GPIO11 are typically used to interface with the flash memory ICs on most esp8266 modules, applying interrupts to these pins are likely to cause problems

Following https://people.eecs.berkeley.edu/~boser/courses/49_sp_2019/N_gpio.html#_interrupts and https://github.com/DL1CB/RamsbergWeatherStation

In [ ]:
anemometer_count = 0

In [ ]:
DEBOUNCE_MS = const(50)

class pulseCount:

    def __init__(self, pin, callback=None, falling=True):  
        """ Count anemometer switches with debouncing. Arguments:
        pin: configured pin (incl PULL_UP/DOWN)
        callback: handler, called when hall switch press detected
        falling: detect raising or falling edges
        """
        self.last_time_ms = 0
        self.detected = False  # a switch press was detected
        self.cb = callback
        pin.irq(self._irq_cb, pin.IRQ_FALLING if falling else pin.IRQ_RISING)

    def pressed(self):     
        """Return True if switch pressed since last call"""
        p = self.detected
        self.detected = False
        return p

    def _irq_cb(self, pin):    
        t = time.ticks_ms()
        diff = t - self.last_time_ms
        if abs(diff) < DEBOUNCE_MS:     
            return
        self.last_time_ms = t
        self.detected = True
        if self.cb: self.cb(pin)   

In [ ]:
def anemometer_reporter(pin):
    global anemometer_count
    anemometer_count = anemometer_count + 1

In [ ]:
anemometer_pin = machine.Pin(14, mode=machine.Pin.IN, pull=machine.Pin.PULL_UP)   
anemometer_detect = pulseCount(anemometer_pin, callback=anemometer_reporter) 

Per the data sheet "A wind speed of 2.4km/h causes the switch to close once per second." 

In [ ]:
def windSpeedKilometersPerHour(pulses, period): 
    """ kmh instantaneous wind speed """
    return round(2.4 * (pulses / period), 4)

## Rain meter

Interrupts may be attached to any GPIO pin except GPIO16. Since GPIO6-GPIO11 are typically used to interface with the flash memory ICs on most esp8266 modules, applying interrupts to these pins are likely to cause problems

Following https://people.eecs.berkeley.edu/~boser/courses/49_sp_2019/N_gpio.html#_interrupts and https://github.com/DL1CB/RamsbergWeatherStation

In [ ]:
rain_gauge_count = 0

In [ ]:
def rain_gauge_reporter(pin):
    global rain_gauge_count
    rain_gauge_count = rain_gauge_count + 1

In [ ]:
rain_gauge_pin = machine.Pin(12, mode=machine.Pin.IN, pull=machine.Pin.PULL_UP)   
rain_detect = pulseCount(rain_gauge_pin, callback=rain_gauge_reporter) 

The rain gauge is a self-emptying tipping bucket type. Each **0.2794mm** of rain causes one momentary contact closure.

In [ ]:
def readData():
    global anemometer_count
    global rain_gauge_count
    wind_direction_raw = wind_direction_pin.read()
    raw_weather_data = {"wind_direction_raw" : wind_direction_raw,
                        #"wind_direction_cardinal" : getDirection(wind_direction_dict, wind_direction_raw),
                        "wind_speed_raw" : anemometer_count,
                        #"wind_speed_kph" : windSpeedKilometersPerHour(anemometer_count, period),
                        "rain_amount_raw" : rain_gauge_count
                        #"rain_amount_ml" : countsToMilliliters(rain_gauge_count)
                       }
    #Reset the anemometer and rain gauge counts
    anemometer_count = 0
    rain_gauge_count = 0
    return raw_weather_data

In [ ]:
readData()

# All together

In [ ]:
import machine
import utime

wind_direction_pin = machine.ADC(0)

class pulseCount:

    DEBOUNCE_MS = const(50)
    def __init__(self, pin, callback=None, falling=True):  
        #Count anemometer switches with debouncing. Arguments:
        #pin: configured pin (incl PULL_UP/DOWN)
        #callback: handler, called when hall switch press detected
        #falling: detect raising or falling edges
        self.last_time_ms = 0
        self.detected = False  # a switch press was detected
        self.cb = callback
        pin.irq(self._irq_cb, pin.IRQ_FALLING if falling else pin.IRQ_RISING)

    def pressed(self):     
        #Return True if switch pressed since last call
        p = self.detected
        self.detected = False
        return p

    def _irq_cb(self, pin):    
        t = utime.ticks_ms()
        diff = t - self.last_time_ms
        if abs(diff) < DEBOUNCE_MS:     
            return
        self.last_time_ms = t
        self.detected = True
        if self.cb: self.cb(pin)  
        
anemometer_count = 0
anemometer_pin = machine.Pin(14, mode=machine.Pin.IN, pull=machine.Pin.PULL_UP)   

def anemometer_reporter(pin):
    global anemometer_count
    anemometer_count = anemometer_count + 1

anemometer_detect = pulseCount(anemometer_pin, callback=anemometer_reporter) 

rain_gauge_count = 0
rain_gauge_pin = machine.Pin(12, mode=machine.Pin.IN, pull=machine.Pin.PULL_UP)   

def rain_gauge_reporter(pin):
    global rain_gauge_count
    rain_gauge_count = rain_gauge_count + 1

rain_detect = pulseCount(rain_gauge_pin, callback=rain_gauge_reporter) 

reset_time = utime.ticks_ms()

def read_wind_rain_data():
    global anemometer_count
    global rain_gauge_count
    global reset_time
    wind_rain_data = {"wind_direction_raw" : wind_direction_pin.read(),
                      "wind_speed_raw" : anemometer_count,
                      "rain_amount_raw" : rain_gauge_count,
                      "elapsed_time" : utime.ticks_diff(utime.ticks_ms(), reset_time)}
    anemometer_count = 0
    rain_gauge_count = 0
    reset_time = utime.ticks_ms()
    return wind_rain_data

In [ ]:
read_wind_rain_data()

In [ ]:
read_wind_rain_data()

In [ ]:
from collections.deque import deque

data_queue = deque()

data_queue.append(read_wind_rain_data())
data_queue.popleft()

In [ ]:
sensors_script = """import machine
import utime

wind_direction_pin = machine.ADC(0)

class pulseCount:

    DEBOUNCE_MS = const(50)
    def __init__(self, pin, callback=None, falling=True):  
        #Count anemometer switches with debouncing. Arguments:
        #pin: configured pin (incl PULL_UP/DOWN)
        #callback: handler, called when hall switch press detected
        #falling: detect raising or falling edges
        self.last_time_ms = 0
        self.detected = False  # a switch press was detected
        self.cb = callback
        pin.irq(self._irq_cb, pin.IRQ_FALLING if falling else pin.IRQ_RISING)

    def pressed(self):     
        #Return True if switch pressed since last call
        p = self.detected
        self.detected = False
        return p

    def _irq_cb(self, pin):    
        t = utime.ticks_ms()
        diff = t - self.last_time_ms
        if abs(diff) < DEBOUNCE_MS:     
            return
        self.last_time_ms = t
        self.detected = True
        if self.cb: self.cb(pin)  
        
anemometer_count = 0
anemometer_pin = machine.Pin(14, mode=machine.Pin.IN, pull=machine.Pin.PULL_UP)   

def anemometer_reporter(pin):
    global anemometer_count
    anemometer_count = anemometer_count + 1

anemometer_detect = pulseCount(anemometer_pin, callback=anemometer_reporter) 

rain_gauge_count = 0
rain_gauge_pin = machine.Pin(12, mode=machine.Pin.IN, pull=machine.Pin.PULL_UP)   

def rain_gauge_reporter(pin):
    global rain_gauge_count
    rain_gauge_count = rain_gauge_count + 1

rain_detect = pulseCount(rain_gauge_pin, callback=rain_gauge_reporter) 

reset_time = utime.ticks_ms()

def read_wind_rain_data():
    global anemometer_count
    global rain_gauge_count
    global reset_time
    wind_rain_data = {"wind_direction_raw" : wind_direction_pin.read(),
                      "wind_speed_raw" : anemometer_count,
                      "rain_amount_raw" : rain_gauge_count,
                      "elapsed_time" : utime.ticks_diff(utime.ticks_ms(), reset_time)}
    anemometer_count = 0
    rain_gauge_count = 0
    reset_time = utime.ticks_ms()
    return wind_rain_data
"""

In [ ]:
import os
print(os.listdir())

In [ ]:
with open('wind_rain_sensors.py') as f:
    print(f.read())

In [ ]:
with open('wind_rain_sensors.txt', 'w') as f:
    f.write(sensors_script)

### Double check the write

In [ ]:
with open('wind_rain_sensors.txt') as f:
    print(f.read())

## Move temp file to `boot.py`

In [ ]:
os.rename('wind_rain_sensors.txt', 'wind_rain_sensors.py')

In [ ]:
machine.reset()

In [ ]:
%disconnect